***` Keras Tuner- Decide Number of Hidden Layers And Neuron In Neural Network`***

This project is basically used to determine Air Quality Index

In [1]:
%reset -f


%pip install pandas tensorflow scikit-learn keras-tuner --quiet


import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from keras_tuner import RandomSearch


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 3.8 MB/s eta 0:00:00


In [2]:
print(f"Pandas Version: {pd.__version__}")
print(f"TensorFlow Version: {tf.__version__}")
print(f"Keras Version: {keras.__version__}")

# Keras Tuner doesn't have a direct __version__ attribute. We'll use the package metadata instead
import pkg_resources
keras_tuner_version = pkg_resources.get_distribution("keras-tuner").version
print(f"Keras Tuner Version: {keras_tuner_version}")

Pandas Version: 2.2.2
TensorFlow Version: 2.18.0
Keras Version: 3.8.0
Keras Tuner Version: 1.4.7


<ipython-input-2-a04ac578cb43>:6: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
  import pkg_resources


In [3]:
# test whether GPU is working or not

tf.test.gpu_device_name()

''

In [4]:
# which CPU/GPU are using

from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 6580997319592661401
 xla_global_id: -1]

In [5]:
# RAM information

!cat /proc/meminfo

MemTotal:       13290452 kB
MemFree:         8355360 kB
MemAvailable:   11741536 kB
Buffers:          407044 kB
Cached:          3115436 kB
SwapCached:            0 kB
Active:           847384 kB
Inactive:        3771556 kB
Active(anon):       3412 kB
Inactive(anon):  1097044 kB
Active(file):     843972 kB
Inactive(file):  2674512 kB
Unevictable:           8 kB
Mlocked:               8 kB
SwapTotal:             0 kB
SwapFree:              0 kB
Dirty:              6144 kB
Writeback:             0 kB
AnonPages:       1096624 kB
Mapped:           714668 kB
Shmem:              3856 kB
KReclaimable:     193260 kB
Slab:             238440 kB
SReclaimable:     193260 kB
SUnreclaim:        45180 kB
KernelStack:        5936 kB
PageTables:        16516 kB
SecPageTables:         0 kB
NFS_Unstable:          0 kB
Bounce:                0 kB
WritebackTmp:          0 kB
CommitLimit:     6645224 kB
Committed_AS:    3378888 kB
VmallocTotal:   34359738367 kB
VmallocUsed:       12296 kB
VmallocChunk:    

In [6]:
# CPU information

!cat /proc/cpuinfo

processor	: 0
vendor_id	: GenuineIntel
cpu family	: 6
model		: 79
model name	: Intel(R) Xeon(R) CPU @ 2.20GHz
stepping	: 0
microcode	: 0xffffffff
cpu MHz		: 2199.998
cache size	: 56320 KB
physical id	: 0
siblings	: 2
core id		: 0
cpu cores	: 1
apicid		: 0
initial apicid	: 0
fpu		: yes
fpu_exception	: yes
cpuid level	: 13
wp		: yes
flags		: fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush mmx fxsr sse sse2 ss ht syscall nx pdpe1gb rdtscp lm constant_tsc rep_good nopl xtopology nonstop_tsc cpuid tsc_known_freq pni pclmulqdq ssse3 fma cx16 pcid sse4_1 sse4_2 x2apic movbe popcnt aes xsave avx f16c rdrand hypervisor lahf_lm abm 3dnowprefetch invpcid_single ssbd ibrs ibpb stibp fsgsbase tsc_adjust bmi1 hle avx2 smep bmi2 erms invpcid rtm rdseed adx smap xsaveopt arat md_clear arch_capabilities
bugs		: cpu_meltdown spectre_v1 spectre_v2 spec_store_bypass l1tf mds swapgs taa mmio_stale_data retbleed bhi
bogomips	: 4399.99
clflush size	: 64
cache_alignment	: 64
ad

In [7]:
try:
    df = pd.read_csv('Real_Combine.csv').dropna()
    X = df.iloc[:, :-1].values  # independent features
    Y = df.iloc[:, -1].values   # dependent features
except Exception as e:
    print(f"Data loading failed: {str(e)}")
    raise

In [8]:
df.head()

,T,TM,Tm,SLP,H,VV,V,VM,PM 2.5
0,7.4,9.8,4.8,1017.6,93.0,0.5,4.3,9.4,219.720833
1,7.8,12.7,4.4,1018.5,87.0,0.6,4.4,11.1,182.187500
2,6.7,13.4,2.4,1019.4,82.0,0.6,4.8,11.1,154.037500
3,8.6,15.5,3.3,1018.7,72.0,0.8,8.1,20.6,223.208333
4,12.4,20.9,4.4,1017.3,61.0,1.3,8.7,22.2,200.645833


In [9]:
df.describe()

,T,TM,Tm,SLP,H,VV,V,VM,PM 2.5
count,1092.000000,1092.000000,1092.000000,1092.000000,1092.000000,1092.000000,1092.000000,1092.000000,1092.000000
mean,26.019963,32.494414,19.468040,1008.070513,62.892857,2.003480,6.753297,15.809432,109.090984
std,7.232026,6.670017,7.437543,7.523290,15.693982,0.747784,3.842442,7.310394,84.465790
min,6.700000,9.800000,0.000000,991.500000,20.000000,0.300000,0.400000,1.900000,0.000000
25%,19.300000,27.800000,12.100000,1001.100000,54.000000,1.600000,3.700000,11.100000,41.833333
50%,28.200000,34.250000,21.200000,1008.050000,64.000000,1.900000,6.500000,14.800000,83.458333
75%,31.700000,37.000000,26.000000,1015.000000,74.000000,2.600000,9.100000,18.300000,158.291667
max,38.500000,45.500000,32.700000,1023.200000,98.000000,5.800000,24.400000,57.600000,404.500000


In [10]:
df.isnull().sum()

,0
T,0
TM,0
Tm,0
SLP,0
H,0
VV,0
V,0
VM,0
PM 2.5,0


In [11]:
X

array([[ 7.4,  9.8,  4.8, ...,  0.5,  4.3,  9.4],
       [ 7.8, 12.7,  4.4, ...,  0.6,  4.4, 11.1],
       [ 6.7, 13.4,  2.4, ...,  0.6,  4.8, 11.1],
       ...,
       [13.9, 24.5, 11.4, ...,  0.6,  8.7, 14.8],
       [16.3, 23. ,  9.8, ...,  1.1,  7.4, 16.5],
       [16.3, 23.4,  9. , ...,  1.3,  7.8, 18.3]])

In [12]:
Y

array([219.72083333, 182.1875    , 154.0375    , ..., 169.        ,
       186.04166667, 185.58333333])

# Data Processing and Spitting

In [13]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

X_scaled = StandardScaler().fit_transform(X)


X_train, X_test, Y_train, Y_test = train_test_split(X_scaled, Y, test_size=0.2, random_state=42)

In [14]:
X_scaled

array([[-2.5758334 , -3.4040112 , -1.97306575, ..., -2.01150096,
        -0.63876585, -0.87715776],
       [-2.52049852, -2.96903049, -2.02687159, ..., -1.87771128,
        -0.61272881, -0.64450561],
       [-2.67266943, -2.86403514, -2.29590082, ..., -1.87771128,
        -0.50858065, -0.64450561],
       ...,
       [-1.67664167, -1.19910894, -1.08526931, ..., -1.87771128,
         0.50686392, -0.13814505],
       [-1.34463241, -1.42409896, -1.30049269, ..., -1.20876285,
         0.1683824 ,  0.0945071 ],
       [-1.34463241, -1.36410162, -1.40810438, ..., -0.94118348,
         0.27253056,  0.34084467]])

# Neural Network Model Creation

In [15]:
def build_model(hp):
    model = keras.Sequential()

    # Input layer
    model.add(layers.InputLayer(input_shape=(X_train.shape[1],)))

    # Hidden layers
    for i in range(hp.Int('num_layers', 2, 3)):
        model.add(layers.Dense(
            units=hp.Int(f'units_{i}', 32, 64, step=16),
            activation='relu',
            kernel_initializer='he_normal'
        ))

    # Output layer
    model.add(layers.Dense(1, activation='linear'))

    # Optimizer
    optimizer = keras.optimizers.Adam(
        learning_rate=hp.Choice('lr', [1e-4, 1e-3]),
        clipvalue=1.0
    )

    model.compile(
        optimizer=optimizer,
        loss='mae',
        metrics=['mae']
    )

    return model

# Hyperparameter Tuning with RandomSearch

In [16]:
tuner = RandomSearch(
    build_model,
    objective='val_mae',
    max_trials=4,
    executions_per_trial=1,
    directory='tuning',
    project_name='air_quality_tuning',
    overwrite=True
)

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/input_layer.py:27: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


In [17]:
tuner.search(
    X_train, Y_train,
    epochs=200,
    validation_data=(X_test, Y_test),
    callbacks=[
        tf.keras.callbacks.EarlyStopping(patience=20)
    ],
    verbose=2
)

Trial 4 Complete [00h 00m 59s]
val_mae: 39.50160598754883

Best val_mae So Far: 32.07036590576172
Total elapsed time: 00h 04m 11s


# Getting the best model & Evaluating the Best Model

In [18]:
best_model = tuner.get_best_models(num_models=1)[0]
print("Test MAE:", best_model.evaluate(X_test, Y_test, verbose=0))

/usr/local/lib/python3.11/dist-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 18 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


Test MAE: [32.07036590576172, 32.07036590576172]


In [19]:
tuner.results_summary()

Results summary
Results in tuning/air_quality_tuning
Showing 10 best trials
Objective(name="val_mae", direction="min")

Trial 2 summary
Hyperparameters:
num_layers: 3
units_0: 64
units_1: 64
lr: 0.001
units_2: 32
Score: 32.07036590576172

Trial 3 summary
Hyperparameters:
num_layers: 3
units_0: 32
units_1: 32
lr: 0.0001
units_2: 48
Score: 39.50160598754883

Trial 0 summary
Hyperparameters:
num_layers: 2
units_0: 64
units_1: 64
lr: 0.0001
Score: 41.31557083129883

Trial 1 summary
Hyperparameters:
num_layers: 2
units_0: 32
units_1: 32
lr: 0.0001
Score: 44.5493278503418
